In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai import *
from fastai.vision import *
import torch
from torch import nn
from dsin.ae.data_manager.data_loader import (
    SideinformationImageImageList, ImageSiTuple)
from dsin.ae import config
from dsin.ae.si_ae import SideInformationAutoEncoder
from dsin.ae.si_net import SiNetChannelIn
from dsin.ae.loss_man import LossManager
from dsin.ae.distortions import Distortions, DistTypes
from dsin.ae.kitti_normalizer import ChangeImageStatsToKitti, ChangeState
from dsin.ae import config


In [3]:
config.H_target = 2* 0.3
config.autoencoder_loss_distortion_to_minimize=DistTypes.MS_SSMIM
config.beta

500

In [4]:
class AverageMetric(Callback):
    "Wrap a `func` in a callback for metrics computation."
    def __init__(self, func,name):
        # If it's a partial, use func.func
        #         name = getattr(func,'func',func).__name__
        self.func, self.name = func, name

    def on_epoch_begin(self, **kwargs):
        "Set the inner value to 0."
        self.val, self.count = 0.,0

    def on_batch_end(self, last_output, last_target, **kwargs):
        "Update metric computation with `last_output` and `last_target`."
        if not is_listy(last_target): last_target=[last_target]
        self.count += last_target[0].size(0) # batch size
        X_DEC_IND =1
        val = self.func(last_output[X_DEC_IND], last_target[0])
        self.val += last_target[0].size(0) * val.detach().cpu()

    def on_epoch_end(self, last_metrics, **kwargs):
        "Set the final result in `last_metrics`."
        return add_metrics(last_metrics, self.val/self.count)

In [5]:
config.use_si_flag = SiNetChannelIn.WithSideInformation
config.use_si_flag

<SiNetChannelIn.WithSideInformation: 6>

In [6]:
torch.__version__

'1.5.1'

In [7]:
si_autoencoder = SideInformationAutoEncoder(config.use_si_flag)
path = "../src/dsin/data"
pct= 0.25 #0.0005 #0.25
valid_image_list = SideinformationImageImageList.from_csv(
    path=path, csv_names=["KITTI_general_val.txt"],pct=pct)
train_image_list = SideinformationImageImageList.from_csv(
    path=path, csv_names=["KITTI_general_train.txt"],pct=pct)

image_lists = ItemLists(
    path=path, train=train_image_list, valid=valid_image_list)

# ll = image_lists.label_from_func(lambda x: x)

tfms = None #get_transforms(do_flip=True, max_rotate=0.0)
batchsize = 1
data = (image_lists
        .label_from_func(lambda x: x)
        .transform(tfms, size=(336, 1224), resize_method=ResizeMethod.CROP, tfm_y=True)
        .databunch(bs=batchsize))
learn = Learner(data=data,
                     model=si_autoencoder,
                     opt_func=torch.optim.Adam,
                     loss_func=LossManager(config.use_si_flag),
                     metrics=[AverageMetric(Distortions._calc_dist,"MS_SSIM")])



In [8]:
learn.model.true_tuple_loss_false_just_out = True
learn.model.use_side_infomation = SiNetChannelIn.NoSideInformation
learn.loss_func.use_side_infomation = SiNetChannelIn.NoSideInformation
learn.fit(1, lr=0.0005)
learn.save('0.5-ae-for-si')

epoch,train_loss,valid_loss,MS_SSIM,time
0,413.701965,495.271118,4040.383789,2:01:29


In [ ]:
# learn = load_learner('ae-for-si')
learn.loss_func.bit_cost_loss_value

In [ ]:
# learn.model.true_tuple_loss_false_just_out = True
# learn.model.use_side_infomation = SiNetChannelIn.NoSideInformation
# learn.loss_func.use_side_infomation = SiNetChannelIn.NoSideInformation
# out =learn.model(learn.data.train_ds[0][0].img.data.unsqueeze(0).cuda()
#                   ,learn.data.train_ds[0][0].si_img.data.unsqueeze(0).cuda())
# out[1]

In [ ]:
# learn.load('mssim -lr0p0005 -full-stage-1')

# learn.model.true_tuple_loss_false_just_out = False

# learn.show_results(figsize=(30,30))

In [ ]:
learn.load('0.5-ae-for-si')
learn.model.true_tuple_loss_false_just_out = True
learn.model.use_side_infomation = SiNetChannelIn.WithSideInformation #SiNetChannelIn.NoSideInformation
learn.loss_func.use_side_infomation = SiNetChannelIn.WithSideInformation # SiNetChannelIn.NoSideInformation
learn.fit(1, lr=0.0005)
learn.save('si-0.5-ae-for-si')

epoch,train_loss,valid_loss,MS_SSIM,time


In [ ]:
# Image(learn.model.my_tuple[0].squeeze_()/255)
learn.model.my_tuple[0]

In [ ]:
Image(learn.model.my_tuple[0].squeeze_())

In [ ]:
learn.model.true_tuple_loss_false_just_out = False

learn.show_results(figsize=(30,30))

In [ ]:
# learn.model.true_tuple_loss_false_just_out = True

# learn.fit(1, lr=0.004)

In [ ]:
learn.model.x_dec

In [ ]:
# learn.load('mssim -lr0p0005 -full-stage-1')
# learn.model.true_tuple_loss_false_just_out = True

# learn.fit(1, lr=0.0005,wd=0.0001)
# learn.save('mssim -lr0p00005 -full-stage-2')

In [ ]:
# learn.fit(1, lr=0.0005,wd=0.0001)

In [ ]:
# learn.load('mssim -lr0p0005 -full-stage-1')

# learn.model.true_tuple_loss_false_just_out = False

# learn.show_results(figsize=(30,30))

In [ ]:

# learn.model.true_tuple_loss_false_just_out = True

# for i in range(4):
#     lr = 0.0001
#     wd = 0.0001
#     learn.fit(1, lr=lr,wd=wd)
# #     learn.save(f'mssim -full-stage-{i} -lr{lr} -wd{wd}')

In [ ]:
# # my_learner.fit_one_cycle(10)
# # my_learner.lr_find()
# # my_learner.save('stage-1')
# # my_learner.recorder.plot()
# # my_learner.load('stage-1')
# # my_learner.save("trained_model-stage1", return_path=True)
# # my_learner.export("trained_model_ex_stage1.pkl")